In [ ]:
import numpy as np

In [ ]:
n = 2
values_received = []
round_messages = {}         # {value: count} 

QUESTION_MARK = "?"
WAITING_MESSAGE = "waiting"
HALF_PLUS_ONE = int(np.floor(n/2)) + 1

In [ ]:
def broadcast(epoch, round, message):
    pass    
def receive(epoch, round, message=None):
    pass
def majority_value():       # use round_messages dict to find the majority value
    pass
def get_messages_count():   # store in round_messages the count per every value
    pass
def quantum_coin_flip():
    return n*"1"

In [ ]:
def agreement(input):
    current = input
    next = False
    epoch = 0
    while True:
        epoch += 1
        broadcast(epoch, 1, current)
        receive(epoch, 1)
        
        get_messages_count()
        current = majority_value()

        broadcast(epoch, 2, current)
        receive(epoch, 2)

        get_messages_count()
        most_frequent_val = max(round_messages, key=round_messages.get)
        answer = None
        if most_frequent_val != QUESTION_MARK:
            answer = most_frequent_val
        number = round_messages.get(most_frequent_val)

        broadcast(epoch, 3, WAITING_MESSAGE)
        receive(epoch, 3, WAITING_MESSAGE)

        coin = quantum_coin_flip()

        if next: 
            break
        
        if number >= HALF_PLUS_ONE:
            current = answer
            next = True
        elif number >= 1:
            current = answer
        else:
            current = coin
    return current